# Интерфейсы как соглашения

До сих пор мы рассматривали только конкретные типы данных. Мы всегда знаем, что из себя представляет конкретный тип данных и что с ним можно делать. В Go есть и другой тип данных, называемый **интерфейсным типом**. Интерфейсы по-другому называют **абстрактными типами**. Интерфейсный тип не раскрывает внутреннюю структуру своих значений или множество операций, которые поддерживает, а лишь предоставляет методы. Когда у нас есть значение интерфейсного типа мы не знаем, что он из себя представляет; мы только знаем что с ним можно делать, а точнее, какое поведение предоставляется его методами.

Интерфейсы можно рассматривать как соглашения. Если некий тип удовлетворяет интерфейсу, то значения такого типа можно использовать ввезде, где используется этот интерфейс. Рассмотрим например функции Printf и Sprintf из пакета fmt. Эти функции являются лишь обёртками над более обобщённой функцией Fprintf. Fprintf имеет следующую сигнатуру:

In [ ]:
func Fprintf(w io.Writer, format string, a ...interface{}) (n int, err error)

Первым аргументом этой функции является интерфейс Writer. Этот интерфейс имеет следующий вид:

In [ ]:
type Writer interface {
    Write(p []byte) (n int, err error)
}

Любой тип, который определяет метод Write именно с такой сигнатурой, которая определена в этом интерфейсе, удовлетворяет интерфейсу Writer, а значит, значения такого типа можно передать функции Fprintf. Именно так и работают функции Printf и Sprintf: Printf передаёт значение типа \*os.File, а Sprintf - \*bytes.Buffer. Оба этих типа удовлетворяют интерфейсу Writer(почему необходимо использовать именно указатели мы узнаем позже). Можно придумать и свой тип, который удовлетворяет Writer'у:

In [3]:
import "fmt"

type ByteCounter int

func (c *ByteCounter) Write(p []byte) (int, error) {
    *c += ByteCounter(len(p))
    return len(p), nil
}

var c ByteCounter
fmt.Fprintf(&c, "Brand New %s", "World")
fmt.Sprint(c)

15


Есть также и другой очень распространённый интерфейс, который позволяет выводить свои типы данных с использованием функций пакета fmt. Это интерфейс Stringer:

In [ ]:
type Stringer interface {
    String() string
}

Рассмотрим пример:

In [5]:
type Color uint32

func (c *Color) String() string {
    return fmt.Sprintf("R:%X, G:%X, B:%X, A:%X", (*c >> 24) & 0xFF, (*c >> 16) & 0xFF, (*c >> 8) & 0xFF, *c & 0xFF)
}

var c Color
c = 0xAABBCCFF
fmt.Sprint(&c)

R:AA, G:BB, B:CC, A:FF
